# Time Modelling Tests

Brendan Smithyman | February, 2016

Test for solvers, plotting wavefields and outputting to `.utout`.

In [2]:
import os
import sys

sys.path.append('../../')
sys.path.append('/usr/local/bin')

import numpy as np
from pygeo.segyread import SEGYFile
from pygeo.fullpy import readini
# from pymatsolver import MumpsSolver
from zephyr.backend import MiniZephyrHD, EurusHD, SparseKaiserSource
from zephyr.middleware import Helm2DViscoProblem, Helm2DSurvey, FullwvDatastore
from zephyr.middleware import dftreal, idftreal, dwavelet, TimeMachine

In [3]:
fds = FullwvDatastore('xhlayr')
systemConfig = fds.systemConfig

systemConfig.update({
    'Disc':     MiniZephyrHD,
#     'Solver':   MumpsSolver,
})

TM = TimeMachine(systemConfig)
STF = TM.fSource(TM.keuper())
systemConfig['sterms'] = STF.ravel()

problem = Helm2DViscoProblem(systemConfig)
survey  = Helm2DSurvey(systemConfig)
problem.pair(survey)

In [ ]:
print('System Wrapper:\t%s'%problem.SystemWrapper)
print('Discretization:\t%s'%problem.system.Disc)
print('RHS Generator: \t%s'%survey.rhsGenerator.__class__)

System Wrapper:	<class 'zephyr.backend.distributors.ViscoMultiFreq'>
Discretization:	<class 'zephyr.backend.minizephyr.MiniZephyrHD'>
RHS Generator: 	<class 'zephyr.backend.source.SparseKaiserSource'>


In [ ]:
uF = problem.fields()[:]['u']

In [ ]:
uFs = uF[:,0,:]
res = TM.idft(uFs)

In [ ]:
%pylab inline
imshow(res[:,20].reshape((200,100)), cmap=cm.bwr)

In [ ]:
%%time

print('Running %(nfreq)d frequencies and %(nsrc)s sources'%{'nfreq': survey.nfreq, 'nsrc': survey.nsrc})

dPred = survey.dpred().reshape((survey.nrec, survey.nsrc, survey.nfreq))

In [ ]:
fds.utoutWrite(dPred)